In [ ]:
import random
import pandas as pd
import numpy as np

from utils.timefeatures import time_features

import random

from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

: 

In [ ]:
df_FCR = pd.read_csv('/home/sepidehfatemi/MissTSM_FCR/FCR_SAITS.csv')
df_FCR

: 

In [ ]:
type_map = {'train': 0, 'val': 1, 'test': 2}
flag = 'train'
set_type = type_map[flag]

scaler = StandardScaler()
df_raw =pd.read_csv('/home/sepidehfatemi/MissTSM_FCR/FCR_SAITS.csv')

'''
df_raw.columns: ['date', ...(other features), target feature]
'''
target = "daily_median_chla_interp_ugL"
cols = list(df_raw.columns)
cols.remove(target)
cols.remove('Date')
df_raw = df_raw[['Date'] + cols + [target]]
# print(cols)
# df_raw
num_train = int(len(df_raw) * 0.7)
num_test = int(len(df_raw) * 0.2)
num_vali = len(df_raw) - num_train - num_test

seq_len = 21

border1s = [0, num_train - seq_len, len(df_raw) - num_test - seq_len]
border2s = [num_train, num_train + num_vali, len(df_raw)]
border1 = border1s[set_type]
border2 = border2s[set_type]


features='MS'
if features == 'M' or features == 'MS':
    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]
elif features == 'S':
    df_data = df_raw[[target]]


scale = True
if scale:
    train_data = df_data[border1s[0]:border2s[0]]
    scaler.fit(train_data.values)
    # print(scaler.mean_)
    # exit()
    data = scaler.transform(df_data.values)
else:
    data = df_data.values

df_stamp = df_raw[['Date']][border1:border2]
df_stamp['Date'] = pd.to_datetime(df_stamp.Date)


timeenc = 0
if timeenc == 0:
    df_stamp['year'] = df_stamp.Date.apply(lambda row: row.year, 1)
    df_stamp['month'] = df_stamp.Date.apply(lambda row: row.month, 1)
    df_stamp['day'] = df_stamp.Date.apply(lambda row: row.day, 1)
    df_stamp['weekday'] = df_stamp.Date.apply(lambda row: row.weekday(), 1)
    # df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
    data_stamp = df_stamp.drop(['Date'], axis=1).values
elif timeenc == 1:
    data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=freq)
    data_stamp = data_stamp.transpose(1, 0)

data_x = data[border1:border2]
data_y = data[border1:border2]

: 

In [ ]:
index=0
s_begin = index
label_len = 7
pred_len = 14
seq_len = 21

s_end = s_begin + seq_len
r_begin = s_end - label_len
r_end = r_begin + label_len + pred_len

seq_x = data_x[s_begin:s_end]
seq_y = data_y[r_begin:r_end]
seq_x_mark = data_stamp[s_begin:s_end]
seq_y_mark = data_stamp[r_begin:r_end]

: 

In [ ]:
len(data_x)

: 

In [ ]:
len(seq_x)

: 

In [ ]:
seq_x_mark

: 

In [ ]:
import numpy as np
def MAE(pred, true):
    mask = ~np.isnan(true)
    pred_filtered = pred[mask]
    true_filtered = true[mask]
    return np.mean(np.abs(pred_filtered - true_filtered))

def MSE(pred, true):
    # Create a mask to identify non-missing values (non-NaN)
    mask = ~np.isnan(true)
    print(mask)
    # Apply the mask to filter out the missing values
    pred_filtered = pred[mask]
    true_filtered = true[mask]
    print(pred_filtered)
    # Compute the MSE only for the non-missing values
    return np.mean((pred_filtered - true_filtered) ** 2)
# Example usage
pred = np.array([1, 2, 3, 4])
true = np.array([2, 2, np.nan, np.nan])

# df_true = pd.read_csv('/home/sepidehfatemi/MissTSM_FCR/df_FCR_missing.csv').drop(columns="Date")
# df_pred = pd.read_csv('/home/sepidehfatemi/MissTSM_FCR/FCR_SAITS.csv').drop(columns="Date")

# pred = df_pred.values
# true = df_true.values

mse = MSE(pred, true)
print("MSE:", mse)


: 

: 

In [ ]:
df_pred

: 